# Climatological Lagrangian Coherent Structure

Objective: Having climatological fields calculate the cLCSs for a region. 
Climatological LCS based on the code developed by Rodrigo Duran for [Matlab](https://bitbucket.org/rodu/clcss/src/master/)

Dec - 2023 Mireya M. Montano mireya.mmor@gmail.com

## Extracting Data.

As part of the Moana Project the data for the Bay of Plenty (region of study) is available through [THREDDS](http://thredds.moanaproject.org:6443/thredds/catalog/moana/catalog.html) server.

In [2]:
import xarray
import cLCS
from cLCS.utils import *